# Исследование базы данных сервиса Stackoverflow

[Схема базы данных и описание базы](https://code.s3.yandex.net/SQL%20for%20data%20and%20analytics/ER/advanced_sql_project.pdf)

**Описание проекта**

StackOverflow похож на социальную сеть — пользователи сервиса задают вопросы, отвечают на посты, оставляют комментарии и ставят оценки другим ответам.
В этом проекте идёт работа с базой данных, которая хранит данные о постах за 2008 год.

**Задача 1**

Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

In [ ]:
SELECT COUNT(post_type_id)
FROM stackoverflow.posts AS p 
JOIN stackoverflow.post_types AS pt ON p.post_type_id=pt.id
WHERE type ='Question'
AND (score>300 OR favorites_count>=100)

**Задача 2**

Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.

In [ ]:
SELECT ROUND(AVG (cnt_questions))
FROM
    (SELECT DATE_TRUNC('day', creation_date)::date,
           COUNT(post_type_id) AS cnt_questions
    FROM stackoverflow.posts AS p 
    JOIN stackoverflow.post_types AS pt ON p.post_type_id=pt.id
    WHERE DATE(creation_date) BETWEEN '2008-11-01' AND '2008-11-18'
    AND type ='Question'
    GROUP BY DATE_TRUNC('day', creation_date)::date) questions

**Задача 3**

Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

In [ ]:
SELECT 
    COUNT(DISTINCT u.id)
FROM stackoverflow.badges AS b
JOIN stackoverflow.users AS u 
    ON b.user_id=u.id
    AND b.creation_date::date=u.creation_date::date

**Задача 4**

Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

In [ ]:
SELECT COUNT(DISTINCT v.post_id)
FROM stackoverflow.votes AS v
JOIN stackoverflow.posts AS p ON v.post_id=p.id
WHERE p.user_id IN (SELECT id
                FROM stackoverflow.users
                WHERE display_name = 'Joel Coehoorn')
AND v.creation_date IS NOT NULL

**Задача 5**

Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.

In [ ]:
SELECT *,
       RANK() OVER(ORDER BY id DESC)
FROM stackoverflow.vote_types
ORDER BY id

**Задача 6**

Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [ ]:
SELECT user_id,
       COUNT(vote_type_id) AS  cnt
FROM stackoverflow.votes AS v
JOIN stackoverflow.vote_types AS vt ON v.vote_type_id=vt.id
WHERE name = 'Close'
GROUP BY user_id
ORDER BY cnt DESC
LIMIT 10

**Задача 7**

Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.
Отобразите несколько полей:
- идентификатор пользователя;
- число значков;
- место в рейтинге — чем больше значков, тем выше рейтинг.

Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.
Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [ ]:
SELECT user_id,
       COUNT(id) AS cnt,
       DENSE_RANK() OVER (ORDER BY COUNT(id) DESC)
FROM stackoverflow.badges
WHERE DATE(creation_date) BETWEEN '2008-11-15' AND '2008-12-15'
GROUP BY user_id
ORDER BY cnt DESC,
         user_id
LIMIT 10

**Задача 8**

Сколько в среднем очков получает пост каждого пользователя?
Сформируйте таблицу из следующих полей:
- заголовок поста;
- идентификатор пользователя;
- число очков поста;
- среднее число очков пользователя за пост, округлённое до целого числа.

Не учитывайте посты без заголовка, а также те, что набрали ноль очков.

In [ ]:
SELECT title,
       user_id,
       score,
       ROUND(AVG(score) OVER (PARTITION BY user_id))
FROM stackoverflow.posts
WHERE title IS NOT NULL
AND score != 0

**Задача 9**

Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

In [ ]:
SELECT p.title
FROM stackoverflow.posts AS p
JOIN stackoverflow.users AS u ON p.user_id = u.id
JOIN stackoverflow.badges AS b ON u.id = b.user_id
WHERE title IS NOT NULL
GROUP BY p.title
HAVING COUNT(b.id) > 1000

**Задача 10**


Напишите запрос, который выгрузит данные о пользователях из Канады (англ. Canada). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:
- пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
- пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
- пользователям с числом просмотров меньше 100 — группу 3.

Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с нулевым количеством просмотров не должны войти в итоговую таблицу.

In [ ]:
SELECT id,
       views,
       (CASE
           WHEN views >= 350 THEN 1
           WHEN views < 350 AND views >= 100 THEN 2
           WHEN views < 100 THEN 3
       END) AS group_views
FROM stackoverflow.users
WHERE location LIKE '%Canada%'
AND views != 0

**Задача 11**

Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [ ]:
SELECT
    id,
    group_views,
    max_views
FROM(
    SELECT DISTINCT
           MAX(views) OVER(PARTITION BY group_views) AS max_views,
           id,
           group_views,
           views
    FROM (
        SELECT 
                id,
                views,
               (CASE
                   WHEN views >= 350 THEN 1
                   WHEN views < 350 AND views >= 100 THEN 2
                   WHEN views < 100 THEN 3
               END) AS group_views
        FROM stackoverflow.users
        WHERE location LIKE '%Canada%'
        AND views != 0) a 
) h
WHERE max_views = views
    ORDER BY max_views DESC, id ASC

**Задача 12**


Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:
- номер дня;
- число пользователей, зарегистрированных в этот день;
- сумму пользователей с накоплением.

In [ ]:
SELECT dt,
       cnt_users,
       SUM(cnt_users) OVER (ORDER BY dt) AS sum_users
FROM(
SELECT EXTRACT ('DAY' FROM creation_date) AS dt,
       COUNT(id) AS cnt_users
FROM stackoverflow.users
WHERE DATE(creation_date) BETWEEN '2008-11-01' AND '2008-11-30'
GROUP BY dt) t

**Задача 13**

Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите:
- идентификатор пользователя;
- разницу во времени между регистрацией и первым постом.

In [ ]:
SELECT p.user_id,
       -(u.creation_date - MIN(p.creation_date))
FROM stackoverflow.users AS u
JOIN stackoverflow.posts AS p ON u.id=p.user_id
GROUP BY p.user_id,
         u.creation_date